<a href="https://colab.research.google.com/github/Dexheimer/CamuCamuEsmaltes/blob/main/Precos_Ceasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurações Iniciais
Criar um arquivo ou célula para as bibliotecas, configurações e funções auxiliares.

In [5]:
# Instalar as bibliotecas necessárias
!pip install pdfplumber requests beautifulsoup4 pandas

# Importar as bibliotecas
import requests
import pandas as pd
import pdfplumber
import os
from bs4 import BeautifulSoup
from datetime import datetime

# configurações iniciais

base_url = "https://www.rj.gov.br/ceasa/node/388"
output_dir = "ceasa_pdfs"
csv_output = "cotacoes_ceasa.csv"



# 2. Coletar Links dos Meses
E testar os links para os meses

In [13]:
def get_month_links(base_url):
    """Acessa a página principal e retorna uma lista de dicionários com links e textos dos meses."""
    response = requests.get(base_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Focar na div específica onde os links estão localizados
    section = soup.find("div", {"property": "schema:text"})

    # Verificar se a seção foi encontrada
    if section:
        links = section.find_all("a", href=True)
        month_links = [
            {"url": link['href'], "text": link.text.strip()}
            for link in links
        ]
        return month_links
    else:
        print("Seção com links não encontrada.")
        return []

# Testar
base_url = "https://www.rj.gov.br/ceasa/node/388"
month_links = get_month_links(base_url)
print(f"Links e textos dos meses encontrados: {month_links}")


Links e textos dos meses encontrados: [{'url': 'https://www.rj.gov.br/ceasa/node/491', 'text': 'Novembro'}, {'url': 'https://www.rj.gov.br/ceasa/node/452', 'text': 'Outubro'}, {'url': 'https://www.rj.gov.br/ceasa/node/446', 'text': 'Setembro'}, {'url': 'https://www.rj.gov.br/ceasa/node/436', 'text': 'Agosto'}, {'url': 'https://www.rj.gov.br/ceasa/node/425', 'text': 'Julho'}, {'url': 'https://www.rj.gov.br/ceasa/node/403', 'text': 'Junho'}, {'url': '/ceasa/node/396', 'text': 'Maio'}, {'url': '/ceasa/node/395', 'text': 'Abril'}, {'url': '/ceasa/node/392', 'text': 'Março'}, {'url': '/ceasa/node/391', 'text': 'Fevereiro'}, {'url': '/ceasa/node/387', 'text': 'Janeiro'}]


# 3. Função para Extrair Links de PDFs

In [17]:
def get_daily_pdf_links(month_link):
    """Acessa uma página do mês e retorna os links para os PDFs diários."""
    response = requests.get(month_link)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todos os links para arquivos PDF
    pdf_links = [
        link['href']
        for link in soup.find_all("a", href=True)
        if link['href'].endswith(".pdf")
    ]
    return pdf_links

"""    # Testar com o primeiro mês
first_month_url = month_links[0]["url"]
daily_pdfs = get_daily_pdf_links(first_month_url)
print(f"PDFs diários encontrados para {month_links[0]['text']}: {daily_pdfs}") """


'    # Testar com o primeiro mês\nfirst_month_url = month_links[0]["url"]\ndaily_pdfs = get_daily_pdf_links(first_month_url)\nprint(f"PDFs diários encontrados para {month_links[0][\'text\']}: {daily_pdfs}") '

# 4. Iterar pelos Links de Todos os Meses.

In [18]:
def get_all_pdf_links(month_links):
    """
    Itera sobre todos os links dos meses e coleta os PDFs diários.
    Retorna um dicionário com o nome do mês e os links dos PDFs.
    """
    all_pdfs = {}
    for month in month_links:
        print(f"Processando: {month['text']} ({month['url']})")
        pdf_links = get_daily_pdf_links(month['url'])
        all_pdfs[month['text']] = pdf_links
        print(f"PDFs encontrados para {month['text']}: {len(pdf_links)}")
    return all_pdfs


In [19]:
# Coletar os links de PDFs para todos os meses
all_pdfs = get_all_pdf_links(month_links)

# Exibir resultado resumido
for month, pdfs in all_pdfs.items():
    print(f"{month}: {len(pdfs)} PDFs encontrados")

Processando: Novembro (https://www.rj.gov.br/ceasa/node/491)
PDFs encontrados para Novembro: 16
Processando: Outubro (https://www.rj.gov.br/ceasa/node/452)
PDFs encontrados para Outubro: 22
Processando: Setembro (https://www.rj.gov.br/ceasa/node/446)
PDFs encontrados para Setembro: 21
Processando: Agosto (https://www.rj.gov.br/ceasa/node/436)
PDFs encontrados para Agosto: 22
Processando: Julho (https://www.rj.gov.br/ceasa/node/425)
PDFs encontrados para Julho: 22
Processando: Junho (https://www.rj.gov.br/ceasa/node/403)
PDFs encontrados para Junho: 20
Processando: Maio (/ceasa/node/396)


MissingSchema: Invalid URL '/ceasa/node/396': No scheme supplied. Perhaps you meant https:///ceasa/node/396?

**Ajustar as urls problemáticas**

In [20]:
def get_all_pdf_links(month_links):
    """
    Itera sobre todos os links dos meses e coleta os PDFs diários.
    Retorna um dicionário com o nome do mês e os links dos PDFs.
    """
    all_pdfs = {}
    base_url = "https://www.rj.gov.br"  # URL base para corrigir links relativos
    for month in month_links:
        # Corrigir URL relativa para URL completa
        month_url = month['url']
        if month_url.startswith('/'):  # Se a URL for relativa
            month_url = base_url + month_url

        print(f"Processando: {month['text']} ({month_url})")
        pdf_links = get_daily_pdf_links(month_url)
        all_pdfs[month['text']] = pdf_links
        print(f"PDFs encontrados para {month['text']}: {len(pdf_links)}")
    return all_pdfs


In [21]:
# Coletar os links de PDFs para todos os meses
all_pdfs = get_all_pdf_links(month_links)

# Exibir resultado resumido
for month, pdfs in all_pdfs.items():
    print(f"{month}: {len(pdfs)} PDFs encontrados")

Processando: Novembro (https://www.rj.gov.br/ceasa/node/491)
PDFs encontrados para Novembro: 16
Processando: Outubro (https://www.rj.gov.br/ceasa/node/452)
PDFs encontrados para Outubro: 22
Processando: Setembro (https://www.rj.gov.br/ceasa/node/446)
PDFs encontrados para Setembro: 21
Processando: Agosto (https://www.rj.gov.br/ceasa/node/436)
PDFs encontrados para Agosto: 22
Processando: Julho (https://www.rj.gov.br/ceasa/node/425)
PDFs encontrados para Julho: 22
Processando: Junho (https://www.rj.gov.br/ceasa/node/403)
PDFs encontrados para Junho: 20
Processando: Maio (https://www.rj.gov.br/ceasa/node/396)
PDFs encontrados para Maio: 20
Processando: Abril (https://www.rj.gov.br/ceasa/node/395)
PDFs encontrados para Abril: 20
Processando: Março (https://www.rj.gov.br/ceasa/node/392)
PDFs encontrados para Março: 17
Processando: Fevereiro (https://www.rj.gov.br/ceasa/node/391)
PDFs encontrados para Fevereiro: 17
Processando: Janeiro (https://www.rj.gov.br/ceasa/node/387)
PDFs encontrados